In [1]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from isodate import parse_duration
import pyspark.sql.types as t
from typing import Tuple
import os

In [2]:
@f.udf(returnType=t.IntegerType())
def converter_segundos(tempo: str) -> int:
    """Converte para o total de seguntos

    Args:
        tempo (str): tempo

    Returns:
        int: total segundos
    """
    duracao = parse_duration(tempo)
    total_minutes = duracao.total_seconds() 
   
    return int(total_minutes)


In [3]:
spark = SparkSession.builder.appName('tratamento_camada_ouro').getOrCreate()
spark

23/11/01 23:37:48 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/11/01 23:37:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/01 23:37:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def carregar_dataframe_prata(spark_session : SparkSession, assunto: str, metrica: str = None):
    if metrica is not None:
        df_cities_skylines = spark_session \
            .read \
                .parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/{assunto}/extracao_data_*/{metrica}/*.parquet')
    else:
        df_cities_skylines = spark_session \
            .read \
                .parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/{assunto}/extracao_data_*/*.parquet')
    return df_cities_skylines

In [41]:
def criar_particao_ouro(dataframe: DataFrame, assunto_pesquisa: str, metrica: str,  nome_arquivo: str, colunas_particao: Tuple = None | str):
    caminho_datalake_ouro = '/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/'
    if colunas_particao == None:
        dataframe.write. \
            parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))
    else: 
        if len(colunas_particao) > 1:
            dataframe.write. \
                partitionBy(*colunas_particao). \
                parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))
        else:
            dataframe.write. \
                partitionBy(colunas_particao). \
                parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))


In [6]:
def transformacao_dataframe(dataframe_transform: DataFrame):
    dataframe_transform = dataframe_transform.select(
        'data_extracao',
        f.explode('DATA_PUBLICACAO').alias('DATA_PUBLICACAO'),
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'ID_VIDEO',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                f.explode('ID_CANAL').alias('ID_CANAL'),
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                f.explode('NM_CANAL').alias('NM_CANAL'), 
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                f.explode('ID_CATEGORIA').alias('ID_CATEGORIA'),
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                f.explode('ID_VIDEO').alias('ID_VIDEO'),
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                f.explode('TITULO_VIDEO').alias('TITULO_VIDEO'),
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'ID_VIDEO',
                f.explode('DESCRICAO').alias('DESCRICAO'),
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                f.explode('TAGS').alias('TAGS'),
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                f.explode('DURACAO_VIDEOS').alias('DURACAO_VIDEOS'),
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                f.explode('TOTAL_VISUALIZACOES').alias('TOTAL_VISUALIZACOES'),
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                f.explode('TOTAL_LIKES').alias('TOTAL_LIKES'),
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                f.explode('TOTAL_FAVORITOS').alias('TOTAL_FAVORITOS'),
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                f.explode('TOTAL_COMENTARIOS').alias('TOTAL_COMENTARIOS'),
        )
    dataframe_transform  = dataframe_transform.na.drop('all')
    dataframe_transform = dataframe_transform.withColumn("data_hora_extracao", f.col('data_extracao'))
    dataframe_transform = dataframe_transform.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd"))
    dataframe_transform = dataframe_transform.withColumn('DURACAO_VIDEO_SEGUNDOS', converter_segundos(f.col('DURACAO_VIDEOS')))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_LIKES', f.col('TOTAL_LIKES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_FAVORITOS', f.col('TOTAL_FAVORITOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_COMENTARIOS', f.col('TOTAL_COMENTARIOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_CARACTERE_VIDEO', f.length('TITULO_VIDEO'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_TAGS', f.when(f.size('TAGS') > 0, f.size('TAGS')).otherwise(0))
    return dataframe_transform
  

In [7]:
@f.udf(returnType=t.StringType())
def traduzir_dia_da_semana(dia):
    traducao = {
        'Sunday': 'Domingo',
        'Monday': 'Segunda-feira',
        'Tuesday': 'Terça-feira',
        'Wednesday': 'Quarta-feira',
        'Thursday': 'Quinta-feira',
        'Friday': 'Sexta-feira',
        'Saturday': 'Sábado'
    }
    return traducao.get(dia, dia)

In [8]:
@f.udf(returnType=t.IntegerType())
def indice_semana(dia):
    traducao = {
        'Domingo': 1,
        'Segunda-feira': 2,
        'Terça-feira': 3,
        'Quarta-feira': 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado': 7
    }
    return traducao.get(dia, dia)

In [9]:
traduzir_dia_da_semana('Sunday')

Column<'traduzir_dia_da_semana(Sunday)'>

In [10]:
df_estatisticas_teste = carregar_dataframe_prata(spark_session=spark, assunto='assunto_cities_skylines', metrica='estatisticas')
df_estatisticas_teste = transformacao_dataframe(df_estatisticas_teste)
df_estatisticas_teste.sort('ID_VIDEO').show(truncate=False)

+-------------+--------------------+------------------------+---------------+------------+-----------+---------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|data_extracao|DATA_PUBLICACAO     |ID_CANAL                |NM_CANAL       |ID_CATEGORIA|ID_VIDEO   |TITULO_VIDEO                                                                     |DESCRICAO                                           

<p>Depara Canal Vídeo</p>

In [11]:
def construir_depara_canal(dataframe: DataFrame, assunto: str):
    dataframe.select(
        dataframe.ID_CANAL,
        dataframe.NM_CANAL,
        dataframe.ID_VIDEO,
        dataframe.TITULO_VIDEO
    ).distinct().sort('ID_CANAL').write.\
        parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_{assunto}/depara_canal.parquet')


<p>O Número de comentários e likes gera mais views? </p>

In [12]:
def gerar_relacao_like_comentario_view_favoritos(dataframe: SparkSession):
    df_comentarios_likes = dataframe.select(
        dataframe.data_hora_extracao,
        dataframe.data_extracao,
        dataframe.ID_CANAL,
        dataframe.NM_CANAL,
        dataframe.ID_VIDEO,
        dataframe.TITULO_VIDEO,
        dataframe.TAGS,
        dataframe.TOTAL_VISUALIZACOES,
        dataframe.TOTAL_LIKES,
        dataframe.TOTAL_FAVORITOS,
        dataframe.TOTAL_COMENTARIOS
    ).filter(
        # (df_estatisticas.data_extracao == '2023-10-27') &
        (dataframe.TOTAL_VISUALIZACOES > 0) &
        (dataframe.TOTAL_LIKES > 0) 
    ).sort(f.col('TOTAL_VISUALIZACOES').desc())
    return df_comentarios_likes


In [13]:
lista_assunto = [
        'Power BI',
        'Python AND dados',
        'Cities Skylines',
        'Cities Skylines 2'
    ]

<p>ETL PARA OBTER RELAÇÃO LIKES - COMENTÁRIOS - VÍDEOS</p>

In [17]:

for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas')
    
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
    criar_particao_ouro(dataframe=df_estatisticas_etl, 
                        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='relacao_viusalicacao_comentario_likes',
        colunas_particao=('data_extracao', 'ID_CANAL', 'ID_VIDEO'),
        nome_arquivo='relacao_viusalicacao_comentario_likes.parquet',
    )
    construir_depara_canal(dataframe=df_estatisticas_etl, assunto=assunto)

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


<p>Dia de publicação</p>

<p>Públicações por semana</p>

In [14]:
def obter_total_publicacao_video(dataframe : DataFrame) -> DataFrame:
   dataframe =  dataframe.select(
    'DATA_PUBLICACAO',
    'ID_VIDEO'
    ) \
        .sort('ID_VIDEO') \
        .withColumn(
            'SEMANA_TRADUZIDA', 
            traduzir_dia_da_semana(
                f.date_format('DATA_PUBLICACAO', 'EEEE')
            )
            ).distinct() \
                .groupBy('SEMANA_TRADUZIDA'). \
                    count(). \
                        alias('TOTAL_VIDEOS'). \
                            withColumn('INDICE_SEMANA', indice_semana('SEMANA_TRADUZIDA')) \
                    .sort('INDICE_SEMANA')\
                        .select('SEMANA_TRADUZIDA', f.col('TOTAL_VIDEOS.COUNT').alias('TOTAL_VIDEOS'))       
   return dataframe

In [15]:
obter_total_publicacao_video(df_estatisticas_teste).show()

+----------------+------------+
|SEMANA_TRADUZIDA|TOTAL_VIDEOS|
+----------------+------------+
|         Domingo|          39|
|   Segunda-feira|          43|
|     Terça-feira|          51|
|    Quarta-feira|          41|
|    Quinta-feira|          49|
|     Sexta-feira|          37|
|          Sábado|          26|
+----------------+------------+



<p>ETL Para obter o total de vídeos públicado por semaba</p>

In [20]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas'
        )
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
    df_estatisticas_etl = obter_total_publicacao_video(df_estatisticas_etl)
    criar_particao_ouro(
        dataframe=df_estatisticas_etl, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='total_video_publicado_semana',
        nome_arquivo='total_video_publicado_semana.parquet',
    )
    
    

ETL PARA  Power BI


ETL PARA  Python AND dados
ETL PARA  Cities Skylines
ETL PARA  Cities Skylines 2


<p>Dia da semana com mais visualização</p>

In [15]:
df_estatisticas_teste.show()

+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|data_extracao|     DATA_PUBLICACAO|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|   ID_VIDEO|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS| data_hora_extracao|DURACAO_VIDEO_SEGUNDOS|TOTAL_CARACTERE_VIDEO|TOTAL_TAGS|
+-------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------------------+----------------------+---------------------+----------+
|   2023-10-27|2023-10-24T00

In [16]:
@f.udf(returnType=t.StringType())
def semana(indice: int) -> str:
    semana = {
        1: 'Domingo',
        2: 'Segunda-feira',
        3: 'Terça-feira',
        4: 'Quarta-feira',
        5: 'Quinta-feira',
        6: 'Sexta-feira',
        7: 'Sábado'
    }
    return semana.get(indice, indice)
    

<p>Vísualizações por semana</p>

In [24]:
df_v_teste = df_estatisticas_teste\
    .select('data_extracao', 'ID_CANAL','ID_VIDEO', 'TOTAL_VISUALIZACOES') \
    .withColumn('INDICE_SEMANA', f.dayofweek('data_extracao'))  \
    .withColumn('SEMANA', semana('INDICE_SEMANA')) \
    .groupBy('SEMANA', 'ID_CANAL', 'ID_VIDEO')\
        .max('TOTAL_VISUALIZACOES')\
        .sort('SEMANA')\
        .withColumnRenamed('max(TOTAL_VISUALIZACOES)', 'MAX_TOTAL_VISUALIZACOES')

df_v_testev2 = df_v_teste.withColumn('ISEMANA', indice_semana('SEMANA')) \
        .groupBy('ISEMANA', 'SEMANA') \
        .sum('MAX_TOTAL_VISUALIZACOES') \
        .sort('ISEMANA') \
    .withColumnRenamed('sum(MAX_TOTAL_VISUALIZACOES)', 'TOTAL_VISUALIZACOES')
df_v_testev2 = df_v_testev2.drop('ISEMANA')
df_v_testev2.show()

+-------------+-------------------+
|       SEMANA|TOTAL_VISUALIZACOES|
+-------------+-------------------+
|      Domingo|             525028|
|Segunda-feira|             670037|
|  Terça-feira|             799358|
| Quarta-feira|             928146|
| Quinta-feira|            1076002|
|  Sexta-feira|            1185093|
|       Sábado|             366474|
+-------------+-------------------+



In [17]:
def obter_total_visualizacoes_semana(dataframe: DataFrame) -> DataFrame:
    df_v_teste = dataframe\
        .select('data_extracao', 'ID_CANAL','ID_VIDEO', 'TOTAL_VISUALIZACOES') \
        .withColumn('INDICE_SEMANA', f.dayofweek('data_extracao'))  \
        .withColumn('SEMANA', semana('INDICE_SEMANA')) \
        .groupBy('SEMANA', 'ID_CANAL', 'ID_VIDEO')\
            .max('TOTAL_VISUALIZACOES')\
            .sort('SEMANA')\
            .withColumnRenamed('max(TOTAL_VISUALIZACOES)', 'MAX_TOTAL_VISUALIZACOES')

    df_v_testev2 = df_v_teste.withColumn('ISEMANA', indice_semana('SEMANA')) \
            .groupBy('ISEMANA','ID_CANAL','ID_VIDEO', 'SEMANA') \
            .sum('MAX_TOTAL_VISUALIZACOES') \
            .sort('ISEMANA') \
        .withColumnRenamed('sum(MAX_TOTAL_VISUALIZACOES)', 'TOTAL_VISUALIZACOES')
    df_v_testev2 = df_v_testev2.drop('ISEMANA')
    return df_v_testev2


In [18]:
obter_total_visualizacoes_semana(df_estatisticas_teste).show()

+--------------------+-----------+-------+-------------------+
|            ID_CANAL|   ID_VIDEO| SEMANA|TOTAL_VISUALIZACOES|
+--------------------+-----------+-------+-------------------+
|UCvEj9EurcC8q5hkV...|IWZZ9ejWCRA|Domingo|                 39|
|UCkXA2EoaXRFW18vr...|zEHZpgrv3D8|Domingo|                 83|
|UCDGo_s0I-2JpO3o3...|fTeqTuEHs7I|Domingo|              14582|
|UCElNoZcxhVqDtZBS...|t4pfQW9O3Lw|Domingo|               2834|
|UCTrNnXivZU_1-aJq...|WxbGu6RTIWg|Domingo|                 10|
|UCayiH2Y5JBEOSubq...|FK-EYFdPqpA|Domingo|               3349|
|UCi9Uzh14BHqkN7UO...|fSw3Cf2Olx0|Domingo|               1283|
|UCayiH2Y5JBEOSubq...|RKa9PtQbAGQ|Domingo|              29665|
|UC7v51W2NoLFawizg...|U3e8h9OgBEE|Domingo|                  6|
|UC7v51W2NoLFawizg...|Ls3dVzGc4_E|Domingo|                 14|
|UCrOH1V-FyMunBIMr...|up4KrHrRwLY|Domingo|              26784|
|UCwBN6GQiolq3EkRP...|I8PSpoki4T0|Domingo|                144|
|UCABm0SdJCoM6z0MF...|TL9F-IPreyU|Domingo|             

<p>ETL Para visializações por semana<P>

In [ ]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_estatisticas_etl = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='estatisticas'
        )
    df_estatisticas_etl = transformacao_dataframe(df_estatisticas_etl)
    df_estatisticas_etl = obter_total_visualizacoes_semana(df_estatisticas_etl)
    criar_particao_ouro(
        dataframe=df_estatisticas_etl, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='total_visualizacao_por_semana',
        colunas_particao=('ID_CANAL', 'ID_VIDEO'),
        nome_arquivo='total_visualizacao_por_semana.parquet',
    )

23/11/01 21:26:47 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1053)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:238)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java

<p>Comentários</p>

In [8]:
df_comentarios = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_power_bi', 
        metrica='comentarios'
)

In [19]:
def extrair_comentarios(dataframe: DataFrame) -> DataFrame:
    dataframe = dataframe \
        .select('ID_CANAL', 'ID_COMENTARIO', 'ID_VIDEO', 'TEXTO_COMENTARIO')  \
        .distinct().sort('ID_VIDEO') 
    return dataframe
# Ugw-AOQ6Kh-Qof2U9bd4AaABAg - Gl60rzPP9VQ

In [20]:
extrair_comentarios(df_comentarios).show(truncate=False)

NameError: name 'df_comentarios' is not defined

<p>Extacao comentário</p>

In [55]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()
    df_comentarios = carregar_dataframe_prata(
        spark_session=spark, 
        assunto=f'assunto_{assunto}', 
        metrica='comentarios'
    )
    df_comentarios = extrair_comentarios(df_comentarios)
    criar_particao_ouro(
        dataframe=df_comentarios, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='comentarios',
        colunas_particao=('ID_CANAL', 'ID_VIDEO'),
        nome_arquivo='comentarios.parquet',
    )

ETL PARA  Power BI


ETL PARA  Python AND dados


ETL PARA  Cities Skylines


ETL PARA  Cities Skylines 2


In [42]:
for assunto in lista_assunto:
    print('ETL PARA ', assunto)
    assunto = assunto.replace(' ', '_').lower()

    df_resposta_comentarios = carregar_dataframe_prata(
        assunto=f'assunto_{assunto}',
        spark_session=spark,
        metrica='resposta_comentarios'
    )   


    df_resposta_comentarios = df_resposta_comentarios.select(
        'ID_CANAL', 
        'ID_RESPOSTA_COMENTARIOS', 
        'TEXTO').withColumnRenamed(
            'ID_CANAL', 'ID_CANAL'
        ).distinct()
    

    # df_resposta_comentarios = df_resposta_comentarios \
    #     .select('ID_CANAL', 'ID_RESPOSTA_COMENTARIOS', 'TEXTO')\
    #     .withColumnRenamed('CABAK', "IDCANAL") \
    #     .distinct()
    # df_resposta_comentarios.show()
    criar_particao_ouro(
        dataframe=df_resposta_comentarios, 
        assunto_pesquisa=f'assunto_{assunto}', 
        metrica='reposta_comentarios',
        colunas_particao=('ID_CANAL', 'ID_RESPOSTA_COMENTARIOS'),
        nome_arquivo='reposta_comentarios.parquet',
    )

ETL PARA  Power BI
+--------------------+-----------------------+--------------------+
|            ID_CANAL|ID_RESPOSTA_COMENTARIOS|               TEXTO|
+--------------------+-----------------------+--------------------+
|UC9LwSTLQx2V7G8wm...|   Ugwhxl0no2RAeQil5...|Que bom que gosto...|
|UCRIafgU7_URCZXt6...|   Ugwj1EtcG0mvXTAD8...|Fixamos o link no...|
|UCcinq1ymmAmhFZzy...|   UgwL2fH9cbxK2r1rS...|Morar em outro es...|
|UCcuLEJUZy4jo9iFv...|   UgwsolFxkM_Atqem2...|excelente resulta...|
|UC5K-TQsItHnNLjqY...|   UgyI2VHceNpu7WIXB...|@Renan Crepaldi m...|
|UC5K-TQsItHnNLjqY...|   Ugy6NsljS1kOqH_hq...|          😅😂😂😂😂|
|UCcuLEJUZy4jo9iFv...|   Ugzr3P1gMOLzP1toL...|Sucesso nessa nov...|
|UCMdNR-VCYnozREcE...|   UgzqUvA2WxBkhxPcK...|Show de bola 👏 ?...|
|UC_Fk7hHbl7vv_7K8...|   Ugy0n2RTLhGkN99mN...|Tic tic nervoso.....|
|UCNaIAElKPANu-d7y...|   UgxZBrxtWkAf27eGv...|            Valeu 👊|
|UCcuLEJUZy4jo9iFv...|   Ugzz8fu7K2qg7troM...|mostre clientes q...|
|UCkSLAYplobH42Vri...|   UgwYcB7KTz3

ETL PARA  Python AND dados
+--------------------+-----------------------+--------------------+
|            ID_CANAL|ID_RESPOSTA_COMENTARIOS|               TEXTO|
+--------------------+-----------------------+--------------------+
|UCI0hxWIdIqSuX_Mc...|   UgwJgDTz0smwkdqNc...|ahhh entendi amor...|
|UCI0hxWIdIqSuX_Mc...|   UgxDKoO0iXTAmVQeV...|Sim! Eu tive prob...|
|UCHQun4keJGrj8RqD...|   UgzF4-RDweWxeADBj...|Tanto esse como o...|
|UCHQun4keJGrj8RqD...|   UgxMnYH4aeHhxl_wL...|                  🤪|
|UCHQun4keJGrj8RqD...|   UgxP-lSfHC_bYd2z6...|Amém irmão uhauah...|
|UCHQun4keJGrj8RqD...|   Ugyp3FWBeoXDVQhpE...|É só para não adm...|
|UCHQun4keJGrj8RqD...|   UgzyQ483ipFD1QaxX...|                  Up|
|UCxKAuN76I7bkAK0s...|   UgwSW7i2QtMp9_a71...|Jerry sempre em a...|
|UCHQun4keJGrj8RqD...|   UgyDh7eOpm_CzAYXH...|Infelizmente muit...|
|UCbww3yb1rObMQURa...|   UgzslhvQKbXLci09y...|Seloco!! Responsa...|
|UCHQun4keJGrj8RqD...|   UgyyEs8p6jn4c-FAs...|@NarizHunter é me...|
|UC_Fk7hHbl7vv_7K8...|

ETL PARA  Cities Skylines
+--------------------+-----------------------+--------------------+
|            ID_CANAL|ID_RESPOSTA_COMENTARIOS|               TEXTO|
+--------------------+-----------------------+--------------------+
|UCrOH1V-FyMunBIMr...|   Ugx8DgeGGf2r7HsrP...|Estacionamentos t...|
|UCrOH1V-FyMunBIMr...|   Ugwtb0MSIh1mdFeTh...|Se n tiver 8 de v...|
|UCrOH1V-FyMunBIMr...|   UgwH8BtufUQtS-tu5...|@Jonas D. não tem...|
|UCOYJm7BsLV8VCkW2...|   UgxLOUCbBhD4HL3sa...|opa vlw pela dica...|
|UCfY4ggUDPeecGSCN...|   Ugw3e2uRx3JdCpOc5...|vai rodar, mas em...|
|UCrOH1V-FyMunBIMr...|   UgyZb7P9u-CJDa1ea...|caaaara, duas pes...|
|UC1mk6EtfMjxR4eEZ...|   Ugxc6yLDT9g3poSTo...|Estamos muito fel...|
|UCEfL_pp3h8Ul-LIL...|   UgwaCYf5yEODk4GRg...|           Ficou sim|
|UCe9jrI0YQ5SM6h5Q...|   Ugy4mQXLiT6PUO3EN...|Na descrição do v...|
|UCDGo_s0I-2JpO3o3...|   UgwUXGu9W-P0HawBD...|Oi Paulo! Bem vin...|
|UCZJCKa3l5khIWKLJ...|   UgwQ8QssIOdGWE2ng...|            😍😍😍😍|
|UCrOH1V-FyMunBIMr...|   U

ETL PARA  Cities Skylines 2
+--------------------+-----------------------+--------------------+
|            ID_CANAL|ID_RESPOSTA_COMENTARIOS|               TEXTO|
+--------------------+-----------------------+--------------------+
|UCrOH1V-FyMunBIMr...|   Ugx8DgeGGf2r7HsrP...|Estacionamentos t...|
|UCrOH1V-FyMunBIMr...|   Ugwtb0MSIh1mdFeTh...|Se n tiver 8 de v...|
|UCrOH1V-FyMunBIMr...|   UgwH8BtufUQtS-tu5...|@Jonas D. não tem...|
|UCOYJm7BsLV8VCkW2...|   UgxLOUCbBhD4HL3sa...|opa vlw pela dica...|
|UCfY4ggUDPeecGSCN...|   Ugw3e2uRx3JdCpOc5...|vai rodar, mas em...|
|UCrOH1V-FyMunBIMr...|   UgyZb7P9u-CJDa1ea...|caaaara, duas pes...|
|UC1mk6EtfMjxR4eEZ...|   Ugxc6yLDT9g3poSTo...|Estamos muito fel...|
|UCEfL_pp3h8Ul-LIL...|   UgwaCYf5yEODk4GRg...|           Ficou sim|
|UCe9jrI0YQ5SM6h5Q...|   Ugy4mQXLiT6PUO3EN...|Na descrição do v...|
|UCDGo_s0I-2JpO3o3...|   UgwUXGu9W-P0HawBD...|Oi Paulo! Bem vin...|
|UCZJCKa3l5khIWKLJ...|   UgwQ8QssIOdGWE2ng...|            😍😍😍😍|
|UCrOH1V-FyMunBIMr...|  

In [62]:
df_resposta_comentarios.select('ID_CANAL', 'ID_RESPOSTA_COMENTARIOS', 'TEXTO').distinct().show(truncate=False)

+------------------------+-------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID_CANAL                |ID_RESPOSTA_COMENTARIOS                          |TEXTO                                                                                                                                                                                                                                                                                                                         |
+------------------------+-------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------

In [73]:
df_resposta_comentarios.printSchema()

root
 |-- ID_CANAL: string (nullable = true)
 |-- ID_RESPOSTA_COMENTARIOS: string (nullable = true)
 |-- TEXTO: string (nullable = true)

